In [16]:
using Revise

using BilevelTrajOpt
using LinearAlgebra
using ForwardDiff

In [4]:
a = rand(4)
A = Matrix(Diagonal(a))

4×4 Array{Float64,2}:
 0.9575  0.0        0.0      0.0      
 0.0     0.0128424  0.0      0.0      
 0.0     0.0        0.19172  0.0      
 0.0     0.0        0.0      0.0898911

In [27]:
# test the solution
function f(x)
    return sum(x)
end

function h(x)
    return [6. .- x[1], x[1]*x[2], x[1]*.5 + x[3] - 4.]
end

function g(x)
    return vcat(-x[2:3],5. - x[4],10. - x[4],17. - x[4],5. - x[1]*x[4])
end

x0 = zeros(4)
λ0 = zeros(length(h(x0)))
μ0 = zeros(length(g(x0)))
c0 = 1.

x_sol, λ_sol, μ_sol, c_sol = auglag_solve(x0,λ0,μ0,f,h,g,c0=c0)

num_h = length(λ0)
num_g = length(μ0)
x_sol_ip = ip_solve(x0,f,h,g,num_h,num_g)

println(x_sol)
println(x_sol_ip)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Solve_Succeeded
[6.0, 5.42607e-14, 1.0, 17.0]
[6.0, 2.16408e-24, 1.0, 17.0]


In [30]:
# test the gradient
function solve_prob(z)
    function f(x)
        return sum(x)
    end

    function h(x)
        return [z[1] - x[1], x[1]*x[2], x[1]*.5 + x[3] - z[2]]
    end

    function g(x)
        return vcat(-x[2:3], z[3] - x[4], 10. - x[4], z[4] - x[4], 5. - x[1]*x[4])
    end

    x0 = zeros(4)
    λ0 = zeros(length(h(x0)))
    μ0 = zeros(length(g(x0)))

    # x_sol_ip = ip_solve(x0,f,h,g,length(h(x0)),length(g(x0)))
    # println(x_sol_ip)
    
    x_sol, λ_sol, μ_sol, c_sol = auglag_solve(x0,λ0,μ0,f,h,g)
    # println(x_sol)

    x_sol
end

solve_prob (generic function with 1 method)

In [38]:
# z0 = [2.,-1.,0.,1.]
z0 = [6.,4.,5.,17.]

sol = solve_prob(z0)

# autodiff 
tic()
J_auto = ForwardDiff.jacobian(solve_prob,z0)
toc()

# numerical
tic()
ϵ = sqrt(eps(1.))
J_num = zeros(size(J_auto))
for i = 1:length(z0)
    δ = zeros(length(z0))
    δ[i] = ϵ
    J_num[:,i] = (solve_prob(z0 + δ) - sol) ./ ϵ
end
toc()

println("----")
display(sol)
display(J_auto)
display(J_num)

elapsed time: 0.022758788 seconds


┌ Warning: `tic()` is deprecated, use `@time`, `@elapsed`, or calls to `time_ns()` instead.
│   caller = top-level scope at In[38]:5
└ @ Core In[38]:5
┌ Warning: `toc()` is deprecated, use `@time`, `@elapsed`, or calls to `time_ns()` instead.
│   caller = top-level scope at In[38]:9
└ @ Core In[38]:9


4-element Array{Float64,1}:
  6.00000000000004      
  5.4260720300866163e-14
  0.9999999999997551    
 17.000000013235137     

4×4 Array{Float64,2}:
  1.0          4.23412e-10  -1.38019e-14  -2.21658e-7
  3.31396e-9  -2.01452e-12   4.83657e-14   1.15114e-9
 -0.5          1.0           6.78428e-14   1.07815e-7
 -2.2183e-7   -2.39202e-9    6.61032e-9    1.0       

4×4 Array{Float64,2}:
  1.00011       5.53131e-5   -3.016e-5     -9.78112e-5 
  1.04434e-5    1.0244e-5    -2.88654e-5    1.55314e-5 
 -0.500132      1.00008      -0.000226647   0.000269547
 -0.000388384  -0.000138044  -0.000286579   0.999758   

elapsed time: 0.027304991 seconds
----


┌ Warning: `tic()` is deprecated, use `@time`, `@elapsed`, or calls to `time_ns()` instead.
│   caller = top-level scope at In[38]:10
└ @ Core In[38]:10
┌ Warning: `toc()` is deprecated, use `@time`, `@elapsed`, or calls to `time_ns()` instead.
│   caller = top-level scope at In[38]:20
└ @ Core In[38]:20
